In [ ]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.12"

In [ ]:
#!import ./../config/Settings.cs

using System;
using Azure;
using Azure.AI.OpenAI;
using System.Net.Http;
using Newtonsoft.Json;

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile("./../config/settings.json");

string endpoint = azureEndpoint;
string key = apiKey;
model = "gpt-4-vision";

In [ ]:
string GPT4V_KEY = key; // Set your key here

string QUESTION = "Please describe the picture";
string IMAGE_PATH = "./Lavaredo.jpeg"; // Set your image path here

//string QUESTION = "How many steps are needed to complete the entire process?";
//string IMAGE_PATH = "./flow.png"; // Set your image path here

string GPT4V_ENDPOINT  = azureEndpoint + @"/openai/deployments/" + model + "/chat/completions?api-version=2023-07-01-preview";

var encodedImage = Convert.ToBase64String(File.ReadAllBytes(IMAGE_PATH));
using (var httpClient = new HttpClient())
{
    httpClient.DefaultRequestHeaders.Add("api-key", GPT4V_KEY);
    var payload = new
    {
        messages = new object[]
        {
            new {
                role = "system",
                content = new object[] {
                    new {
                        type = "text",
                        text = "You are an AI assistant that helps people find information."
                    }
                }
            },
            new {
                role = "user",
                content = new object[] {
                    new {
                        type = "image_url",
                        image_url = new {
                            url = $"data:image/jpeg;base64,{encodedImage}"
                        }
                    },
                    new {
                        type = "text",
                        text = QUESTION
                    }
                }
            }
        },
        temperature = 0.7,
        top_p = 0.95,
        max_tokens = 800,
        stream = false
    };

    var response = await httpClient.PostAsync(GPT4V_ENDPOINT , new StringContent(JsonConvert.SerializeObject(payload), Encoding.UTF8, "application/json"));

    if (response.IsSuccessStatusCode)
    {
        var responseData = JsonConvert.DeserializeObject<dynamic>(await response.Content.ReadAsStringAsync());
        //Console.WriteLine(responseData);
        Console.WriteLine(responseData.choices[0].message.content);
    }
    else
    {
        Console.WriteLine($"Error: {response.StatusCode}, {response.ReasonPhrase}");
    }
}

In [ ]:
// const string rawImageUri = "https://upload.wikimedia.org/wikipedia/commons/8/8c/Drei_Zinnen-Tre_Cime_Di_Lavaredo_6.JPG";

// OpenAIClient client = new(new Uri(endpoint), new AzureKeyCredential(key));

// ChatCompletionsOptions chatCompletionsOptions = new()
// {
//     DeploymentName = model,
//     Messages =
//     {
//         new ChatRequestSystemMessage("You are a helpful assistant that describes images."),
//         new ChatRequestUserMessage(
//             new ChatMessageTextContentItem("Hi! Please describe this image"),
//             new ChatMessageImageContentItem(new Uri(rawImageUri))),
//     },
// };

// Response<ChatCompletions> chatResponse = await client.GetChatCompletionsAsync(chatCompletionsOptions);
// ChatChoice choice = chatResponse.Value.Choices[0];
// if (choice.FinishDetails is StopFinishDetails stopDetails)
// {
//     Console.WriteLine($"{choice.Message.Role}: {choice.Message.Content}");
// }